# 1B

In [2]:
!pip install sentence-transformers pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from scipy.stats import pearsonr
import numpy as np

def preprocess_dataset(file_path):
    processed_rows = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            columns = line.strip().split('\t')
            if len(columns) > 3:
                columns = columns[-3:]
            processed_rows.append(columns)

    df = pd.DataFrame(processed_rows, columns=['score', 'sentence1', 'sentence2'])
    df['score'] = pd.to_numeric(df['score'], errors='coerce')

    return df

def calculate_cosine_similarity(df, model):
    similarities = []
    for _, row in df.iterrows():
        embeddings1 = model.encode(row['sentence1'], convert_to_tensor=True)
        embeddings2 = model.encode(row['sentence2'], convert_to_tensor=True)
        cosine_sim = util.pytorch_cos_sim(embeddings1, embeddings2).item()
        similarities.append(cosine_sim)
    return similarities

train_df = preprocess_dataset('C:\\Users\\Dharani\\Downloads\\train.csv')
dev_df = preprocess_dataset('C:\\Users\\Dharani\\Downloads\\dev.csv')

dev_df = dev_df[dev_df['score'] < 3.5]
model = SentenceTransformer('nli-distilroberta-base-v2')

dev_df['calculated_similarity'] = calculate_cosine_similarity(dev_df, model)
dev_df['calculated_similarity'].fillna(0, inplace=True)

dev_df['scaled_similarity'] = (dev_df['calculated_similarity'] + 1) * 5 / 2

dev_df.replace([np.inf, -np.inf], np.nan, inplace=True)
dev_df.dropna(subset=['score', 'scaled_similarity'], inplace=True)

correlation, _ = pearsonr(dev_df['score'], dev_df['scaled_similarity'])
print(f"Pearson correlation coefficient: {correlation}")

Pearson correlation coefficient: 0.7897237326225071


C:\Users\Dharani\AppData\Local\Temp\ipykernel_22196\1138113549.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dev_df['calculated_similarity'].fillna(0, inplace=True)


In [ ]:
xx = [i for i in dev_df['score'] if i<=5]
print(len(xx))

1495


# 1C

Without dropping

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, util, evaluation
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from scipy.stats import pearsonr


def calculate_cosine_similarity(df, model):
    similarities = []
    for _, row in df.iterrows():
        embeddings1 = model.encode(row['sentence1'], convert_to_tensor=True)
        embeddings2 = model.encode(row['sentence2'], convert_to_tensor=True)
        cosine_sim = util.pytorch_cos_sim(embeddings1, embeddings2).item()
        similarities.append(cosine_sim)
    return similarities

def preprocess_dataset(file_path):
    processed_rows = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            columns = line.strip().split('\t')
            if len(columns) > 3:
                columns = columns[-3:]
            processed_rows.append(columns)

    df = pd.DataFrame(processed_rows, columns=['score', 'sentence1', 'sentence2'])
    df['score'] = pd.to_numeric(df['score'], errors='coerce')

    return df

train_df = preprocess_dataset('/content/train.csv')
dev_df = preprocess_dataset('/content/dev.csv')

train_df = train_df[train_df['score'] <= 5]
dev_df = dev_df[dev_df['score'] <= 5]

train_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['score']) for index, row in train_df.iterrows()]
dev_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['score']) for index, row in dev_df.iterrows()]

model = SentenceTransformer('nli-distilroberta-base-v2')

for name, param in model.named_parameters():
    if not any(layer in name for layer in [ 'layer.5','pooler']):
        param.requires_grad = False

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=model)

evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_examples, batch_size=16, name='dev')

model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=2, evaluation_steps=1800,
          warmup_steps=100)

model.save('/content/sentence_bert_finetuned')

model = SentenceTransformer('/content/sentence_bert_finetuned')

dev_df['calculated_similarity'] = calculate_cosine_similarity(dev_df, model)
dev_df['scaled_similarity'] = (dev_df['calculated_similarity'] + 1) * 5 / 2
dev_df.replace([np.inf, -np.inf], np.nan, inplace=True)
dev_df.dropna(subset=['score', 'scaled_similarity'], inplace=True)

correlation, _ = pearsonr(dev_df['score'], dev_df['scaled_similarity'])
print(f"Pearson correlation coefficient after fine-tuning: {correlation}")

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/359 [00:00<?, ?it/s]

Iteration:   0%|          | 0/359 [00:00<?, ?it/s]

Pearson correlation coefficient after fine-tuning: 0.8094167386810214


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Inference Pipeline for Demo

In [15]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

def load_test_dataset(file_path):
    df = pd.read_csv(file_path, delimiter='\t', quoting=3, encoding='utf-8', names=['sentence1', 'sentence2'])
    return df

def calculate_cosine_similarity(df, model):
    similarities = []
    i=1
    for _, row in df.iterrows():
        embeddings1 = model.encode(row['sentence1'], convert_to_tensor=True)
        embeddings2 = model.encode(row['sentence2'], convert_to_tensor=True)
        cosine_sim = util.pytorch_cos_sim(embeddings1, embeddings2).item()
        similarities.append(cosine_sim)
    return similarities

def generate_predictions_file(df, similarities, output_file='demo.csv'):
    df['similarity'] = similarities
    df['scaled_similarity'] = (df['similarity'] + 1) * 2.5
    df.iloc[1:].to_csv(output_file, columns=['sentence1', 'sentence2', 'scaled_similarity'], index=False)
    print(f"Predictions saved to {output_file}")

model = SentenceTransformer('/content/drive/MyDrive/sentence_bert_finetuned')

test_df = load_test_dataset('/content/sample_test.csv')

similarities = calculate_cosine_similarity(test_df, model)

generate_predictions_file(test_df, similarities)


Predictions saved to demo.csv
